In [ ]:


import torch
import xarray as xr
import glob
from toolz import *

from gnl.calculus import c_grid_advective_tendency
from uwnet.thermo import get_dz

import holoviews as hv
hv.extension('bokeh')

%opts Image [colorbar=True, width=400, height=300](cmap='viridis'){+axiswise +framewise}


class config:
    workdir = "/Users/noah/Data/0/"

def to_normal_units_nn_columns(ds):
    """Convert output from uwnet.columns to have proper SI units"""
    scales = {
        'FQT': 1 / 86400 / 1000,
        'FSL': 1 / 86400,
        'Q1NN': 1 / 86400 / 1000,
        'qt': 1 / 1000,
        'qtOBS': 1 / 1000,
        'Q2NN': 1 / 86400 / 1000,
        'QN': 1/1000,
        'QP': 1/1000
    }

    for key, scale in scales.items():
        if key in ds:
            ds = ds.assign(**{key: ds[key] * scale})

    return ds

def compute_divergence(state):


    u, v, w = get('U V W'.split(), state)
    dx  = float(ds.x[1]-ds.x[0])
    dz = get_dz(ds.z).values
    rho = state['layer_mass'] / dz
    f = np.ones_like(u)

    hdiv = c_grid_advective_tendency(u, v, 0*w, f, dx, dz, rho)
    vdiv = c_grid_advective_tendency(0*u, 0*v, w, f, dx, dz, rho)
    
    return hdiv, vdiv


def compute_advection_tendency(state, key):
    u, v, w = get('U V W'.split(), state)
    dx  = float(ds.x[1]-ds.x[0])
    dz = get_dz(ds.z).values
    rho = state['layer_mass'] / dz
    return  c_grid_advective_tendency(u, v, w, pkl[key], dx, dz, rho)


def mass_integral(x, w):
    y = x * np.reshape(w, (-1, 1, 1))
    return y.sum(axis=-3)


def load_data(id):
    # load data
    
    pattern = config.workdir + id + '*/NG1_test_*.pkl'
    print(pattern)
    pkls = [merge(torch.load(f).values())
            for f in  glob.glob(pattern)]
    ds = xr.open_zarr("../data/training_data.zarr/").pipe(to_normal_units_nn_columns)
    return pkls, ds

In [ ]:
def compare_cwp_id(id):

    pkls, ds = load_data(id)

    pkl = pkls[0]

    cwp = mass_integral(pkl['QN'], pkl['layer_mass'])
    plt.pcolormesh(cwp)
    plt.colorbar()
    plt.figure()
    (ds.QN[0] * ds.layer_mass).sum('z').plot()

In [ ]:
compare_cwp_id("86/462bf2")

The predicted CWP is very different between the simulations. The neural network really struggles to predict this quantity correctly. It does okay in the tropics, but not elsewhere.

In [ ]:
pkls, ds = load_data("c0/d2a5df")
print(len(pkls))

In [ ]:
hmap = hv.HoloMap(kdims=['field', 'y'])


for j in [10, 20 ,32]:
    for key in 'qt sl U V W'.split():
        x = pkls[0][key][:,j,0]
        y = ds[key].isel(y=j,x=0,time=0).values

        elm = hv.Curve(x-y, label='truth')
        hmap[(key, j)] = elm
    
hmap.opts({'Curve': {'norm': dict(framewise=True, axiswise=True)}}).layout('field')

In [ ]:
compare_cwp_id("c0/d2a5df")

the differences are not that big. I am suprised it make such a difference. The error in the temperature is significant I guess.

Okay I need to do more debugging. I should generate a case which makes a very small time step and see

1. Do TABS, QV, QN, QP change at all
2. Are the inputs to the NN identical to the training data.

# Run SAM for 3 time steps with very small time step

The basic state should not change

In [ ]:
from common import *

caseid = "NG1_2018-08-15-A"

def open_ic_and_case(caseid, sam_dir=config.sam):
    """Open initial condition and 3d output files for SAM
    
    Returns
    -------
    ic : initial condition
    ds : 3d output of SAM
    """
    ic = xr.open_dataset(sam_dir + "/NG1/ic.nc")
    ds  = open_case(caseid)
    ic = ic.assign_coords(x=ds.x, y=ds.y)
    
    return ic, ds

In [ ]:
from toolz import curry


def compare_ds_with_ic(ic, ds, field, t):
    return ic[field], ds[field].sel(time=t)


@curry
def image_comparison(ic, ds, field, t, z):
    x,y = compare_ds_with_ic(ic, ds, field, t)
    return hv.Image((y-x).sel(z=z))



def concat_kind(ds, ic):
    common_vars = list(set(ic.data_vars) & set(ds.data_vars))
    return xr.concat((ic[common_vars], ds[common_vars]), dim=pd.Index(['NG', 'SAM'], name='kind'))


def case_visualizer(caseid, t=0):
    ic, ds = open_ic_and_case(caseid)
    ds = ds.isel(time=t)

    return concat_kind(ds, ic).to_array(dim='field')\
    .pipe(hv.Dataset)\
    .to.image(["x", "y"], dynamic=True)

From this it looks like the vapor and condensate fields disagree from the very outset, but that QP and the temperature are pretty close.

We can see that QN is identically zero.

In [ ]:
%%opts Image[width=300, height=200] {+axiswise}
case_visualizer("NG1_2018-08-15-A", t=1).grid(["kind", "field"])

Good these look identical. Now, I can safely call the neural network from within SAM.

# NN in SAM


Calling NN in sam, but there are problems

In [ ]:
def open_debug(path):
    return [torch.load(f)['kwargs'] for f in  glob.glob(f"{path}*/NG1_test_*.pkl")]


def plot_rel_error_z(x, y):
    ans = np.sqrt(((x-y)**2).mean(['x', 'y']))
    ans = ans / np.sqrt((x**2).mean(['x', 'y'])) 
    return hv.Curve(ans)

def norm_3d(x, y, w):
    return float(((x-y)**2*w).mean().pipe(np.sqrt))

def rel_error_3d(x, y, w):
    return norm_3d(x, y, w)/norm_3d(x, 0, w)


id = "90/ad19f6"
pkls= open_debug(config.nextflow_workdir + id)
ic = xr.open_mfdataset(config.nextflow_workdir + id + "*/NG1/ic.nc")
ds = xr.open_zarr("../data/training_data.zarr").sel(time=ic.time).pipe(to_normal_units_nn_columns)

In [ ]:
hmap = hv.HoloMap({key: plot_rel_error_z(ds[key],  pkls[0][key])
                   for key in 'sl qt FQT FSL U V W'.split()})
hmap.layout()

There are very large relative errors in the forcing fields, and smaller errors in the velocity field and temperature field

In [ ]:
def print_rel_error_report(id):
    pkls= open_debug(config.nextflow_workdir + id)
    ic = xr.open_mfdataset(config.nextflow_workdir + id + "*/NG1/ic.nc")
    ds = xr.open_zarr("../data/training_data.zarr").sel(time=ic.time).pipe(to_normal_units_nn_columns)
    
    rel_errors = {key: norm_3d(ds[key], pkls[0][key], ds.layer_mass)
               for key in 'sl qt FQT FSL U V W'.split()}
    return rel_errors

In [ ]:
print_rel_error_report("90/ad19f6")

Is the problem with the velocities, or is it with the way SAM is computing FSL given U, V, W? Let's recompute FQT using the same routine used to get the training data.

In [ ]:
ds.FQT.mean(['x']).plot()

In [ ]:
training_ds = xr.open_zarr("../data/training_data.zarr")

In [ ]:
training_ds.FSL[0].mean(['x']).plot()

In [ ]:
fsl = xr.DataArray(pkls[0]['FSL'], dims=training_ds.FSL[0].dims, coords=training_ds.FSL[0].coords)
fsl.mean(['x']).plot()

this actually looks pretty similar.

In [ ]:
from gnl.calculus import c_grid_advective_tendency
from uwnet.thermo import get_dz


def compute_advection_tendency(state, key, z, x):
    u, v, w = get('U V W'.split(), state)
    dx  = float(x[1]-x[0])
    dz = get_dz(z).values

    rho = state['layer_mass'] / dz
    return  c_grid_advective_tendency(u, v, w, state[key], dx, dz, rho)

FSL_adv = compute_advection_tendency(pkls[0], 'sl', ds.z, ds.x)

In [ ]:
rel_error_3d(fsl, FSL_adv, ds.layer_mass)

In [ ]:
%%opts Image{-axiswise -framewise}

hmap = hv.HoloMap({
    'Computed from advection': hv.Image(FSL_adv[:,32,:]),
    'Training Data': hv.Image(ds.FSL[:,32,:].values),
    'SAM': hv.Image(pkls[0]['FSL'][:,32,:])
})

hmap.layout()

## Domain mean FQT, FSL and W do not vanish

In [ ]:
(86400*ds.FSL.mean(['x', 'y'])).plot()

In [ ]:
plt.plot(86400*FSL_adv.mean(axis=(-1,-2)))

In [ ]:
plt.plot(ds.W.mean(axis=(-2,-1)), label='mu')
# plt.plot(ds.W.std(axis=(-2,-1)), label='sig')
plt.legend()

The domain mean vertical velocity does not vanish in the training data. This is some kind of mistake. This does not seem to be a problem with the coarse-graining. because averaging the high resolution results in a similar domain mean vertical velocity profile.

Is the average vertical velocity closer to zero after one time step with SAM?

In [ ]:
plt.plot(pkls[1]['W'].mean(axis=(-2,-1)))
plt.plot(pkls[0]['W'].mean(axis=(-2,-1)))

The vertical velocity measured with SAM is actually much smaller. Perhaps this is a result of the pressure correction step.

# SAM implements the divergence free condition

Does the coarse-resolution data I have respect SAMs divergence free condition? I could check this by seeing if the SAM changes the velocity field after a very short time step. This is the same experiment as run [above](#Run-SAM-for-3-time-steps-with-very-small-time-step).

In [ ]:
%%opts Curve[invert_axes=True]

caseid = "NG1_2018-08-15-A"
ic, ds = open_ic_and_case(caseid)

hv.Curve(ic.W.mean(['x', 'y']), label='Initial cond.') \
* hv.Curve(ds.W[1].mean(['x', 'y']), label='After 1 timestep')

Indeed we can see that the domain mean vertical velocity vanishes after one time step, which means the pressure correction is active. Maybe `write_fields_3d` happens after the model updates the velocity, and runs the pressure correction step.

# Compute advection tendency using divergent free winds

I can use SAM to produce a divergence free velocity field by taking one very small time step and saving the output. 

In [ ]:
from sam.case import pressure_correct

ic_p = pressure_correct(ic, sam_src=config.sam)

The domain mean vertical velocity of this should vanish

In [ ]:
hv.Curve(ic_p.W.mean(['x', 'y'])[0])\
* hv.Curve(ic.W.mean(['x', 'y']), label='Key')

Computing the advection tendency using this pressure corrected velocity field seems to improve things:

In [ ]:
%%opts  Raster[invert_yaxis=True](cmap='RdBu_r'){-axiswise -framewise}


# compute avection tendency
ic_p['layer_mass'] = training_ds.layer_mass
ic_p_dict = {key: val.values for key, val in ic_p.variables.items()}
ic_p_dict['sl'] = training_ds['sl'][0]
fsl_pressure_corrected = compute_advection_tendency(ic_p_dict, 'sl', training_ds.z, ic_p['x'])

# visualize it
vals = {
     'Computed from advection': FSL_adv[:,32,:],
    'Training Data': training_ds.FSL[0,:,32,:].values/86400,
    'SAM': pkls[0]['FSL'][:,32,:],
    'Pressure Corrected': fsl_pressure_corrected[0,:,32,:]
}


hmap = hv.HoloMap({key: hv.Raster(val, kdims=['x index', 'z index']) for key, val in vals.items()})
hmap.layout().cols(2)

the domain mean of FSL is also much smaller (although not exactly zero).

In [ ]:
%%opts Curve[invert_axes=True]

lay = hv.Curve(fsl_pressure_corrected[0].mean(axis=(-2,-1))*86400, kdims=['z'], vdims=['FSL'], label='Pressure Corrected ') * \
hv.Curve(training_ds.FSL[0].mean(['x', 'y']).values, kdims=['z'], label='Training Data FSL')

lay.redim.unit(FSL='K/day', z='index').relabel("Domain mean FSL") \
+ hv.Curve(training_ds.z.values).redim.label(x='index', y='z').redim.unit(z='m')